In [ ]:
# Importing necessary libraries
import geopandas as gpd
import rasterio.mask
import fiona
import os
import numpy as np
import shapefile
import rasterstats
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import rasterio
import time
import cv2
import math

In [ ]:
from typing import cast, Optional

# Importing necessary programs from the libraries
from rasterio.plot import show
from rasterio.transform import from_origin
from rasterio.crs import CRS
from rasterio import features
from rasterstats import zonal_stats
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt

In [ ]:
import PIL.Image

def img_scale(img: np.ndarray) -> np.ndarray:
    return (img * 255).astype(np.uint8)

def display_image(img: np.ndarray, _: Optional[str] = None):
    display(PIL.Image.fromarray(img))

def display_contours(img: np.ndarray, contours):
    img_with_contours = img.copy()
    for contour in contours:
        cv2.drawContours(img_with_contours, [contour], 0, (0, 255, 0), 2)
    display_image(img_with_contours)

In [ ]:
img = cv2.imread('Test_Images/temp_ndvi.jpg')
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
display_image(img_grey)

In [ ]:
ret, thresh  = cv2.threshold(img_grey, 10, 255, cv2.THRESH_BINARY)
kernel1= np.ones((7,7), np.uint8)
kernel2 = np.ones((5,5), np.uint8)

eroded = cv2.erode(thresh, kernel1, iterations = 1)
dilated = cv2.dilate(eroded, kernel2, iterations = 8)

display_image(dilated)

In [ ]:
edged = cv2.Canny(img_grey, 30, 200)

display_image(edged)

In [ ]:
contours, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

col_grey_img = cv2.cvtColor(img_grey, cv2.COLOR_GRAY2RGB)

print(len(contours))
display_contours(col_grey_img, contours)

In [ ]:
large_contours = [contour for contour in contours if 80 < cv2.contourArea(contour)]

print(len(large_contours))
display_contours(col_grey_img, large_contours)

In [ ]:
contours2, _ = cv2.findContours(img_grey, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

print(len(contours2))
display_contours(col_grey_img, contours2)

In [ ]:
large_contours2 = [contour for contour in contours2 if 200 < cv2.contourArea(contour)]

print(len(large_contours2))
display_contours(col_grey_img, large_contours2)

In [ ]:
def gen_line_img(img: np.ndarray, print_img: np.ndarray = cast(np.ndarray, np.copy(col_grey_img)) * 0) -> tuple[np.ndarray, np.ndarray]:
    """line, img"""

    # see https://stackoverflow.com/a/45560545/8626693

    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 50  # minimum number of pixels making up a line
    max_line_gap = 20  # maximum gap in pixels between connectable line segments
    line_image = np.copy(print_img)

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 5)
    
    return lines, line_image

In [ ]:
lines1, line_img1 = gen_line_img(edged, col_grey_img)
print(len(lines1))
print(sum([len(line) for line in lines1]))
display_image(line_img1)